In [15]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')
import matplotlib.pyplot as plt

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz



# Utils

In [2]:
# Batch normalization class.

class batch_norm(object):
    def __init__(self, epsilon=1e-5, momentum = 0.9, name="batch_norm"):
        with tf.variable_scope(name):
            self.epsilon  = epsilon
            self.momentum = momentum
            self.name = name

    def __call__(self, x, train=True):
        return tf.contrib.layers.batch_norm(x,
                          decay=self.momentum, 
                          updates_collections=None,
                          epsilon=self.epsilon,
                          scale=True,
                          is_training=train,
                          scope=self.name)

# Leaky Relu
def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak*x)

# For projecting latent sample to different size (output size)
def linear(input_, output_size, scope=None, stddev=0.02, bias_start=0.0, with_w=False):
    shape = input_.get_shape().as_list()

    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("Matrix", [shape[1], output_size], tf.float32,
        tf.random_normal_initializer(stddev=stddev))
        bias = tf.get_variable("bias", [output_size],
        initializer=tf.constant_initializer(bias_start))
        if with_w:
            return tf.matmul(input_, matrix) + bias, matrix, bias
        else:
            return tf.matmul(input_, matrix) + bias

# Transposed convolution.      
def deconv2d(input_, output_shape, k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02, name="deconv2d", with_w=False):
    with tf.variable_scope(name):
        # filter : [height, width, output_channels, in_channels]
        w = tf.get_variable('w', [k_h, k_w, output_shape[-1], input_.get_shape()[-1]],
                  initializer=tf.random_normal_initializer(stddev=stddev))
    
        try:
            deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape,
                    strides=[1, d_h, d_w, 1])

        # Support for verisons of TensorFlow before 0.7.0
        except AttributeError:
            deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape,
                    strides=[1, d_h, d_w, 1])

        biases = tf.get_variable('biases', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())

        if with_w:
            return deconv, w, biases
        else:
            return deconv

# Normal convolution.
def conv2d(input_, output_dim, k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02, name="conv2d"):
    with tf.variable_scope(name):
        w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim],
                  initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding='SAME')
        biases = tf.get_variable('biases', [output_dim], initializer=tf.constant_initializer(0.0))
        conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())
        return conv


# Generator

In [3]:
gf_dim = 64 # Dimension of gen filters in first conv layer. [64]
batch_size = 108

def generator(z, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
        g_bn0 = batch_norm(name='g_bn0')
        g_bn1 = batch_norm(name='g_bn1')
        g_bn2 = batch_norm(name='g_bn2')
        g_bn3 = batch_norm(name='g_bn3')
        """   
        z_, h0_w, h0_b = linear(z, gf_dim*8*4*4, 'g_h0_lin', with_w=True)

        h0 = tf.reshape(z_, [-1, 4, 4, gf_dim * 8])
        h0 = tf.nn.relu(g_bn0(h0))

        h1, h1_w, h1_b = deconv2d(h0,
            [batch_size, 8, 8, gf_dim*4], name='g_h1', with_w=True)
        h1 = tf.nn.relu(g_bn1(h1))

        h2, h2_w, h2_b = deconv2d(h1,
            [batch_size, 16, 16, gf_dim*2], name='g_h2', with_w=True)
        h2 = tf.nn.relu(g_bn2(h2))

        h3, h3_w, h3_b = deconv2d(h2,
            [batch_size, 28, 28, gf_dim*1], name='g_h3', with_w=True)
        h3 = tf.nn.relu(g_bn3(h3))

        h4, h4_w, h4_b = deconv2d(h3,
            [batch_size, 28, 28, 1], name='g_h4', with_w=True)
        """
        z_, h0_w, h0_b = linear(z, gf_dim*4*7*7, 'g_h0_lin', with_w=True)

        h0 = tf.reshape(z_, [-1, 7, 7, gf_dim * 4])
        h0 = tf.nn.relu(g_bn0(h0))

        h1, h1_w, h1_b = deconv2d(h0,
            [batch_size, 14, 14, gf_dim*2], name='g_h1', with_w=True)
        h1 = tf.nn.relu(g_bn1(h1))

        h2, h3_w, h3_b = deconv2d(h1,
            [batch_size, 28, 28, 1], name='g_h3', with_w=True)
        h2 = tf.nn.relu(g_bn3(h2))
        return tf.nn.tanh(h2)

#temp = tf.zeros((1,100))
#generator(temp)

# Discriminator

In [4]:
df_dim = 64 # Dimension of discrim filters in first conv layer. [64]

def discriminator(image, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):        
        d_bn1 = batch_norm(name='d_bn1')
        d_bn2 = batch_norm(name='d_bn2')
        d_bn3 = batch_norm(name='d_bn3')

        h0 = lrelu(conv2d(image, df_dim, name='d_h0_conv'))
        h1 = lrelu(d_bn1(conv2d(h0, df_dim*2, name='d_h1_conv')))
        h2 = lrelu(d_bn2(conv2d(h1, df_dim*4, name='d_h2_conv')))
        h3 = lrelu(d_bn3(conv2d(h2, df_dim*8, name='d_h3_conv')))
        h4 = linear(tf.reshape(h3, [-1, 8192]), 1, 'd_h3_lin')

        return tf.nn.sigmoid(h4), h4

#temp = tf.zeros((108,64,64,3))
#discriminator(temp)

In [5]:
tf.reset_default_graph() 
image_dims = [28,28,1]
z = tf.placeholder(tf.float32, [None, 100], name='z')
y = tf.placeholder(tf.float32, [batch_size, 1], name='y')
inputs = tf.placeholder(tf.float32, [batch_size] + image_dims, name='real_images')

G = generator(z)
D, D_logits = discriminator(inputs)
D_, D_logits_ = discriminator(G, reuse=True)

d_loss_real = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logits,
                                            labels=tf.ones_like(D)))
d_loss_fake = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logits_,
                                            labels=tf.zeros_like(D_)))
d_loss = d_loss_real + d_loss_fake

g_loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logits_,
                                            labels=tf.ones_like(D_)))

G_sum = tf.summary.image("G", G)
g_loss_sum = tf.summary.scalar("g_loss", g_loss)
d_loss_sum = tf.summary.scalar("d_loss", d_loss)

In [6]:
t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'd_' in var.name]
g_vars = [var for var in t_vars if 'g_' in var.name]
print(g_vars)

[<tf.Variable 'generator/g_h0_lin/Matrix:0' shape=(100, 12544) dtype=float32_ref>, <tf.Variable 'generator/g_h0_lin/bias:0' shape=(12544,) dtype=float32_ref>, <tf.Variable 'generator/g_bn0/beta:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'generator/g_bn0/gamma:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'generator/g_h1/w:0' shape=(5, 5, 128, 256) dtype=float32_ref>, <tf.Variable 'generator/g_h1/biases:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'generator/g_bn1/beta:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'generator/g_bn1/gamma:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'generator/g_h3/w:0' shape=(5, 5, 1, 128) dtype=float32_ref>, <tf.Variable 'generator/g_h3/biases:0' shape=(1,) dtype=float32_ref>, <tf.Variable 'generator/g_bn3/beta:0' shape=(1,) dtype=float32_ref>, <tf.Variable 'generator/g_bn3/gamma:0' shape=(1,) dtype=float32_ref>]


In [7]:
d_optim = tf.train.AdamOptimizer(0.0002, beta1=0.5).minimize(d_loss, var_list=d_vars)
g_optim = tf.train.AdamOptimizer(0.0002, beta1=0.5).minimize(g_loss, var_list=g_vars)

In [20]:
epochs = 20
#N = mnist.train.num_examples//batch_size
N = 100
with tf.Session() as sess:
    counter = 1
    sess.run(tf.global_variables_initializer())
    g_sum = tf.summary.merge([g_loss_sum, G_sum])
    d_sum = tf.summary.merge([d_loss_sum])    
    writer = tf.summary.FileWriter("./logs", sess.graph)
    for e in range(0,epochs):
        print('='*20 + " Epoch {}\{} ".format(e+1, epochs) + '='*20)
        for i in range(N):
            batch = mnist.train.next_batch(batch_size) # Next mnist batch.            
            batch_mnist = batch[0].reshape((batch_size, 28, 28, 1)) # Flatten.
            batch_mnist = batch_mnist*2 - 1 # Rescale.            
            batch_z = np.random.uniform(-1, 1, size=(batch_size, 100)) # Random noise for generator.
            
            # Run optimizers.
            _ = sess.run(d_loss, feed_dict={inputs: batch_mnist, z: batch_z})
            _ = sess.run(g_loss, feed_dict={z: batch_z})
        
        loss_discriminator, summary_d = sess.run([d_loss, d_sum],{inputs: batch_mnist, z: batch_z})
        loss_generator, summary_g = sess.run([g_loss, g_sum], {z: batch_z})
        print(f'Generator loss: {loss_generator}')
        print(f'Discriminator loss: {loss_discriminator}')
        writer.add_summary(summary_d, counter)
        writer.add_summary(summary_g, counter)
        counter = counter+1
        #print(f'Discriminator loss: {train_loss_d}', end='\n')
        #print(f'Generator loss: {train_loss_g}', end='\n')
        sample_z = np.random.uniform(-1, 1, size=(batch_size, 100))
        generator_img = sess.run(G, feed_dict={z: sample_z})
        img = tf.reshape(generator_img, [-1, 28, 28])

==================== Epoch 1\20 ====================
Generator loss: 1.3482816219329834
Discriminator loss: 1.5394530296325684
==================== Epoch 2\20 ====================
Generator loss: 1.3166136741638184
Discriminator loss: 1.3312227725982666
==================== Epoch 3\20 ====================


KeyboardInterrupt: 

In [ ]:
#fig, ax = plt.plot(figsize=(5,5))
plt.imshow(generator_img[10,:,:,0], cmap='gray')